# 1. Обучение модели pysbot

## 1.1 Проверка наличия карты

In [1]:
!nvidia-smi

Thu Jun 10 09:39:35 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## 1.2 Установка окружения

In [2]:
pip install transformers==4.6.1 #==4.6.1 #pip install transformers==4.4.2 #urllib3==1.25.4 transformers==2.8.0

     |████████████████████████████████| 2.3MB 7.2MB/s 
     |████████████████████████████████| 3.3MB 35.8MB/s 
     |████████████████████████████████| 901kB 64.8MB/s 


In [3]:
%%writefile setup.sh
export CUDA_HOME=/usr/local/cuda-10.1
git clone https://github.com/NVIDIA/apex
cd apex
git reset --hard a651

pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./

Writing setup.sh


In [4]:
!sh setup.sh

Cloning into 'apex'...
remote: Enumerating objects: 8048, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (89/89), done.
remote: Total 8048 (delta 65), reused 93 (delta 41), pack-reused 7913
Receiving objects: 100% (8048/8048), 14.10 MiB | 27.41 MiB/s, done.
Resolving deltas: 100% (5466/5466), done.
HEAD is now at a651e2c sync-free Distributed LAMB + parameter reordering (#1055)
/usr/local/lib/python3.7/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-5t2x8_k9
Created temporary directory: /tmp/pip-req-tracker-qykptkr0
Created requirements tracker '/tmp/pip-req-tracker-qykptkr0'
Created temporary directory: /tmp/pip-install-7ilnef1_
Processing /content/apex
  Created temporary directory: /tmp/pip-req-build-p7s9190v
  Added file

# Датасеты для обучения

## 2.1 Подключаем диск для сохранения модели и скачивания датасета

In [5]:
DATA_PATH='/content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/data'
MODEL_PATH='/content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models'
should_continue=True

In [6]:
# Обеспечиваем подгрузку данных и их хранение в каталоге ноутубка MyDrive/chats_emotions_and_voises/chat04_depression-therapist-chatbot
import os
from google.colab import drive
drive.mount('/content/gdrive')
if not os.path.exists(DATA_PATH):
    raise ValueError('Нет папки для хранения данных', DATA_PATH)
%ls $DATA_PATH


Mounted at /content/gdrive
data.zip  dateset.zip  train-ds210609.zip  train-ds.zip


## 2.2а Продолжение обучения после остановки (с новой виртуальной машины) и скачивания датасета и чекопинат с google.drive
- скачивает чекпоинты
- скачивате обучающий и проверяющий датасет
- **НУЖНО пропустить разделы "Формируем обучающую выборку" 2б**

In [7]:
LAST_CHECKPOINT_ZIP_NAME='checkpoint47.zip'
TRAIN_DATASET_ZIP_NAME='train-ds.zip'
OUTPUT_PATH='/content/comment_model'
should_continue=True

if not os.path.isfile(os.path.join(DATA_PATH, TRAIN_DATASET_ZIP_NAME)): 
  raise Exception(f'Не найден архив "{TRAIN_DATASET_ZIP_NAME}" с сохраненным датасетом в папке "{DATA_PATH}"')
else:
  !unzip $DATA_PATH/$TRAIN_DATASET_ZIP_NAME -d '/content'
  if not os.path.isfile('/content/test.txt') or not os.path.isfile('/content/train.txt'):
    raise Exception(f'Среди распаковыанных не найде обязательный файл датасета')

if not os.path.isfile(os.path.join(MODEL_PATH, LAST_CHECKPOINT_ZIP_NAME)):
  raise Exception(f'Не найден файл чекпоинта "{LAST_CHECKPOINT_ZIP_NAME}" папке "{MODEL_PATH}". Нужно обучать заново или указать другой файл')
else:
  !mkdir $OUTPUT_PATH
  !unzip $MODEL_PATH/$LAST_CHECKPOINT_ZIP_NAME -d $OUTPUT_PATH


Archive:  /content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/data/train-ds.zip
  inflating: /content/train.txt      
  inflating: /content/test.txt       
Archive:  /content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/models/checkpoint47.zip
   creating: /content/comment_model/checkpoint-47000/
  inflating: /content/comment_model/checkpoint-47000/optimizer.pt  
  inflating: /content/comment_model/checkpoint-47000/pytorch_model.bin  
  inflating: /content/comment_model/checkpoint-47000/training_args.bin  
  inflating: /content/comment_model/checkpoint-47000/scheduler.pt  
  inflating: /content/comment_model/checkpoint-47000/merges.txt  
 extracting: /content/comment_model/checkpoint-47000/added_tokens.json  
  inflating: /content/comment_model/checkpoint-47000/tokenizer.json  
  inflating: /content/comment_model/checkpoint-47000/tokenizer_config.json  
  inflating: /content/comment_model/checkpoint-47000/special_tokens_map.json  
  inflating: /content/comment_model/c

## 2.2б Формируем обучающую выборку
(!) Пропускаем, если продолжаем обучение

---



In [ ]:
import os
DATASET_ZIP_NAME='dateset.zip'
if os.path.exists(os.path.join('/content', 'dataset.tsv')):
   data_path = os.path.join('/content', 'dataset.tsv')
elif os.path.exists(os.path.join(DATA_PATH, 'dataset.tsv')):
  data_path = os.path.join(DATA_PATH, 'dataset.tsv')
else:
  if not os.path.isfile(os.path.join(DATA_PATH, TRAIN_DATASET_ZIP_NAME)): 
    raise Exception(f'Не найден архив  "{TRAIN_DATASET_ZIP_NAME}" с сохраненным датасетом в папке "{DATA_PATH}"')
  else:
    !unzip $DATA_PATH/$TRAIN_DATASET_ZIP_NAME -d '/content'
    if not os.path.isfile('/content/dataset.tsv'):
      raise Exception(f'Среди распаковыанных не найде обязательный файл датасета')

In [ ]:
import pandas as pd
import numpy as np
import random


SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
should_continue=False # Если формируем новую выборку, то обучение нужно начинать заново

In [ ]:
data = pd.read_csv(data_path, sep=',', header=None)
print(data.shape)
data.head()

(1270999, 1)


,0
0,"|0|2|Это нормально. Если я не спешу, могу позн..."
1,"|0|3|Всем привет! Ну, этой темы не могло не бы..."
2,"|0|1|Я познакомлюсь, если понравится, нет проб..."
3,|0|3|Подозреваю у себя профессиональное выгора...
4,|0|1|А что же отпуск? Отдохните|1|2|Он только ...


In [ ]:
# Уменьшаем выборку, т.к. у колаба не хватает памяти
# TODO перед тем как резать, надо перемешать
# data = data[:900000] # В коллабе на полном датасете вывалилавается по нехватке памяти в функции load_and_cache_examples при конвертации датасета в токены (память к колабе 26698820)
# 
data_rand=data.sample(frac=1) # Перемешиваем все данные
data = data_rand[:300000] # Выбираем первые записи



In [ ]:
# Формируем выборку обучающую и проверочную
train=data.sample(frac=0.8) #random state is a seed value
test=data.drop(train.index)
print('Длина обучающей', len(train), 'и проверочной выборки', len(test))
print('Пример обучающей выборки')
train.head()

Длина обучающей 240000 и проверочной выборки 60000
Пример обучающей выборки


,0
344552,"|0|1|Жду, что поможете выбрать правильное напр..."
717143,"|0|3|Просто от того, чтомы ДОПУСКАЕМ, что они ..."
187003,|0|2|Я с ним не спала ещё. Я не хочу отношений...
31178,"|0|2|Деньги от тяжелой и изматывающей работы, ..."
80785,"|0|2|Откуда у автора инфа про секретарей, не п..."


In [ ]:
with open('train.txt', 'w') as f:
  for val in train.values:
        f.write(f'{val[0]}\n')
with open('test.txt', 'w') as f:
  for val in test.values:
        f.write(f'{val[0]}\n')

In [ ]:
# TODO сохранение датасет на диске
TRAIN_DATASET_ZIP_NAME='train-ds.zip'

if os.path.isfile(os.path.join(DATA_PATH, TRAIN_DATASET_ZIP_NAME)): 
  raise Exception(f'Уже есть архив с датасетом "{TRAIN_DATASET_ZIP_NAME}" в папке "{DATA_PATH}". Удалите его сначала')
else:
  !zip -r -9 $DATA_PATH/$TRAIN_DATASET_ZIP_NAME './train.txt' './test.txt'

  adding: train.txt (deflated 71%)
  adding: test.txt (deflated 71%)
	zip warning: name not matched: ./content/dataset.tsv

zip error: Nothing to do! (try: zip -r -9 /content/gdrive/MyDrive/chats_emotions_and_voises/psy_chatbot/data/dateset.zip . -i ./content/dataset.tsv)


# 3 Обучаем модель на диалогах (Run finetuning)

## 3.1 Импорт библиотек

In [8]:
# Источник https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
import argparse
import glob
import logging
import os
import pickle

import re
import shutil
from typing import Dict, List, Tuple

import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
from tqdm import tqdm, trange

import transformers
from transformers import (
    MODEL_WITH_LM_HEAD_MAPPING,
    WEIGHTS_NAME,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM, # AutoModelWithLMHead,
    AutoTokenizer,
    PreTrainedModel,
    PreTrainedTokenizer,
    get_linear_schedule_with_warmup,
)
from torch.utils.tensorboard import SummaryWriter

SEED = 1234

logger = logging.getLogger(__name__)

MODEL_CONFIG_CLASSES = list(MODEL_WITH_LM_HEAD_MAPPING.keys())
MODEL_TYPES = tuple(conf.model_type for conf in MODEL_CONFIG_CLASSES)

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

print('Версия torch', torch.__version__)
print('Версия transformers', transformers.__version__)

Версия torch 1.8.1+cu101
Версия transformers 4.6.1


## 3.2 Параметры обучения

In [14]:
# Параметры. Вырезано все для local_rank для распределенного обучения

args = {}
args['should_continue'] = should_continue # Продолжать обучение модели с последнего чекпоинта
args['cache_dir']='model_cache' # Папка для кеширования модели
args['output_dir']='comment_model' # Папка сохранения модели, TODO лучше сохранять на gogle диске, т.к. при остановке колаба удалится #--output_dir=comment_model \
args['model_name_or_path'] = 'sberbank-ai/rugpt3small_based_on_gpt2' # --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
args['train_data_file'] = 'train.txt' # Файл обучающей выборки # --train_data_file=train.txt \
args['eval_data_file'] = 'test.txt' # Валидационный файл # --eval_data_file=valid.txt \
args['per_gpu_train_batch_size'] = 1 # --per_gpu_train_batch_size 1 размер батча на GPU, дефолтное 4, больше - быстрее обучение, ниже качество.
args['max_steps'] = -1 # Максимальное кол-во шагов обучения, переопределяет количество эпох. -1 считать по эпохам.
args['num_train_epochs'] = 5 # --num_train_epochs 5 \
args['gradient_accumulation_steps'] = 1 # --gradient_accumulation_steps 1 Количество накопленных шагов перед переходом градиента
args['learning_rate'] = 5e-5 # The initial learning rate for Adam.
args['weight_decay'] = 0.01 # "Weight decay if we apply some.")
args['adam_epsilon'] = 1e-8 #"Epsilon for Adam optimizer.")
args['max_grad_norm'] = 1.0  #"Max gradient norm.")
args['block_size'] = 2048 # --block_size 2048 \
args['overwrite_output_dir'] = True # --overwrite_output_dir Перезатирать выходную дирректорию
args['logging_steps'] = 500 # Шаги логгирования
args['save_steps'] = 1000 # Сохранение через каждые X шагов
args['save_total_limit'] = 25 # Количество сохраняемых чекопинтов (одна модель в чекпоинте с оптимизатором ~1 Гб)
args['evaluate_during_training'] = False # Вычислять точность по мере обучения - True для вычисления после каждого шага логгирования, но занимает 40 минут примерно
args['per_gpu_eval_batch_size'] = 4 # Batch size per GPU/CPU for evaluation.
# неимпользуемые параметры  --do_train - обучать модель, --do_eval - вычислять результат обучения по валидационной выборке
# --model_type=gpt2 - Тип модели, жестка задан --fp16 - Использование 16битной точности (apex Nvidia)
print(args)

{'should_continue': True, 'cache_dir': 'model_cache', 'output_dir': 'comment_model', 'model_name_or_path': 'sberbank-ai/rugpt3small_based_on_gpt2', 'train_data_file': 'train.txt', 'eval_data_file': 'test.txt', 'per_gpu_train_batch_size': 1, 'max_steps': -1, 'num_train_epochs': 5, 'gradient_accumulation_steps': 1, 'learning_rate': 5e-05, 'weight_decay': 0.01, 'adam_epsilon': 1e-08, 'max_grad_norm': 1.0, 'block_size': 2048, 'overwrite_output_dir': True, 'logging_steps': 500, 'save_steps': 1000, 'save_total_limit': 25, 'evaluate_during_training': False, 'per_gpu_eval_batch_size': 4}


## 3.3 Функции обучения и работы с чепоинтами, вычисления точности

In [15]:
# Возвращение отсортированного списка чекпоинтов моделей
def _sorted_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> List[str]:
    ordering_and_checkpoint_path = []
    glob_checkpoints = glob.glob(os.path.join(args['output_dir'], f"{checkpoint_prefix}-*"))
    for path in glob_checkpoints:
        if use_mtime:
            ordering_and_checkpoint_path.append((os.path.getmtime(path), path))
        else:
            regex_match = re.match(".*{}-([0-9]+)".format(checkpoint_prefix), path)
            if regex_match and regex_match.groups():
                ordering_and_checkpoint_path.append((int(regex_match.groups()[0]), path))
    checkpoints_sorted = sorted(ordering_and_checkpoint_path)
    checkpoints_sorted = [checkpoint[1] for checkpoint in checkpoints_sorted]
    return checkpoints_sorted

def _rotate_checkpoints(args, checkpoint_prefix="checkpoint", use_mtime=False) -> None:
    if not args['save_total_limit']:
        return
    if args['save_total_limit'] <= 0:
        return

    # Check if we should delete older checkpoint(s)
    checkpoints_sorted = _sorted_checkpoints(args, checkpoint_prefix, use_mtime)
    if len(checkpoints_sorted) <= args['save_total_limit']:
        return

    number_of_checkpoints_to_delete = max(0, len(checkpoints_sorted) - args['save_total_limit'])
    checkpoints_to_be_deleted = checkpoints_sorted[:number_of_checkpoints_to_delete]
    for checkpoint in checkpoints_to_be_deleted:
        logger.info(f"Deleting older checkpoint [{checkpoint}] due to args['save_total_limit']")
        shutil.rmtree(checkpoint)

In [16]:
# Вычисление качества обучения
def evaluate(args, model: PreTrainedModel, tokenizer: PreTrainedTokenizer, prefix="") -> Dict:
    # Loop to handle MNLI double evaluation (matched, mis-matched)
    eval_output_dir = args['output_dir']

    eval_dataset = load_and_cache_examples(args, tokenizer, evaluate=True)
    os.makedirs(eval_output_dir, exist_ok=True) # if args.local_rank in [-1, 0]:

    args['eval_batch_size'] = args['per_gpu_eval_batch_size'] * max(1, args['n_gpu'])

    # Note that DistributedSampler samples randomly

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    eval_sampler = SequentialSampler(eval_dataset)
    eval_dataloader = DataLoader(
        eval_dataset, sampler=eval_sampler, batch_size=args['eval_batch_size'], collate_fn=collate
    )

    # multi-gpu evaluate
    if args['n_gpu'] > 1:
        model = torch.nn.DataParallel(model)

    # Eval!
    logger.info("***** Running evaluation {} *****".format(prefix))
    logger.info("  Num examples = %d", len(eval_dataset))
    logger.info("  Batch size = %d", args['eval_batch_size'])
    eval_loss = 0.0
    nb_eval_steps = 0
    model.eval()

    for batch in tqdm(eval_dataloader, desc="Evaluating", position=0, leave=True): #disable=args.local_rank not in [-1, 0] # # For coolab:position=0, leave=True)
        inputs, labels = (batch, batch) # mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch)
        inputs = inputs.to(args['device'])
        labels = labels.to(args['device'])

        with torch.no_grad():
            outputs =  model(inputs, labels=labels) #model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            lm_loss = outputs[0]
            eval_loss += lm_loss.mean().item()
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    perplexity = torch.exp(torch.tensor(eval_loss))

    result = {"perplexity": perplexity}

    output_eval_file = os.path.join(eval_output_dir, prefix, "eval_results.txt")
    with open(output_eval_file, "w") as writer:
        logger.info("***** Eval results {} *****".format(prefix))
        for key in sorted(result.keys()):
            logger.info("  %s = %s", key, str(result[key]))
            writer.write("%s = %s\n" % (key, str(result[key])))

    return result

In [17]:
# Загрузка датасета

#Класс датасета
class TextDataset(Dataset):
    def __init__(self, tokenizer: PreTrainedTokenizer, args, file_path: str, block_size=512):
        assert os.path.isfile(file_path)
        block_size = block_size - (tokenizer.model_max_length - tokenizer.max_len_single_sentence)
        directory, filename = os.path.split(file_path)
        cached_features_file = os.path.join(directory, "gpt2_cached_lm_" + str(block_size) + "_" + filename)

        if os.path.exists(cached_features_file): # and not args.overwrite_cache: (!) убрана логика для перезаписи кешированных файлов альтернативы 
            logger.info("Loading features from cached file %s", cached_features_file)
            with open(cached_features_file, "rb") as handle:
                self.examples = pickle.load(handle)
        else:
            logger.info("Creating features from dataset file at %s", directory)
            self.examples = []
            with open(file_path, encoding="utf-8") as f:
                text = f.read()
            print('Cчитали файл обучающего датасета:', file_path)
            tokenized_text = tokenizer.convert_tokens_to_ids(tokenizer.tokenize(text))
            print('Cконвертировали токены')
            del text # УДАЛЯЕМ загруженный текст, т.к. может нехватать памяти
            for i in range(0, len(tokenized_text) - block_size + 1, block_size):  # Truncate in block of block_size
                self.examples.append(tokenizer.build_inputs_with_special_tokens(tokenized_text[i: i + block_size]))
            # Note that we are loosing the last truncated example here for the sake of simplicity (no padding)
            # If your dataset is small, first you should loook for a bigger one :-) and second you
            # can change this behavior by adding (model specific) padding.
            logger.info("Saving features into cached file %s", cached_features_file)
            with open(cached_features_file, "wb") as handle:
                pickle.dump(self.examples, handle, protocol=pickle.HIGHEST_PROTOCOL)

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item):
        return torch.tensor(self.examples[item], dtype=torch.long)

# Загрузка датасета
def load_and_cache_examples(args, tokenizer, evaluate=False):
    file_path = args['eval_data_file'] if evaluate else args['train_data_file']
    return TextDataset(tokenizer, args, file_path=file_path, block_size=args['block_size'])


In [18]:
# Функция обучения
# Отключено Train with masked-language modeling loss instead of language modeling
def train(args, train_dataset, model: PreTrainedModel, tokenizer: PreTrainedTokenizer) -> Tuple[int, float]:
    """ Train the model """
    tb_writer = SummaryWriter()
    args['train_batch_size'] = args['per_gpu_train_batch_size'] * max(1, args['n_gpu'])

    def collate(examples: List[torch.Tensor]):
        if tokenizer._pad_token is None:
            return pad_sequence(examples, batch_first=True)
        return pad_sequence(examples, batch_first=True, padding_value=tokenizer.pad_token_id)

    train_sampler = RandomSampler(train_dataset) # if args.local_rank == -1 else DistributedSampler(train_dataset)
    train_dataloader = DataLoader(
        train_dataset, sampler=train_sampler, batch_size=args['train_batch_size'], collate_fn=collate
    )

    if args['max_steps'] > 0:
        t_total = args.max_steps
        args['num_train_epochs'] = args['max_steps'] // (len(train_dataloader) // args['gradient_accumulation_steps']) + 1
    else:
        t_total = len(train_dataloader) // args['gradient_accumulation_steps'] * args['num_train_epochs']
    
    model.resize_token_embeddings(len(tokenizer))

    # Prepare optimizer and schedule (linear warmup and decay)
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": args['weight_decay'],
        },
        {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args['learning_rate'], eps=args['adam_epsilon'])
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=0, num_training_steps=t_total # args['warmup_steps'] = 0 количество шагов на прогрев
    )
    # Check if saved optimizer or scheduler states exist
    if (args['model_name_or_path'] and os.path.isfile(os.path.join(args['model_name_or_path'], "optimizer.pt"))
            and os.path.isfile(os.path.join(args['model_name_or_path'], "scheduler.pt"))): # Load in optimizer and scheduler states
        optimizer.load_state_dict(torch.load(os.path.join(args['model_name_or_path'], "optimizer.pt")))
        scheduler.load_state_dict(torch.load(os.path.join(args['model_name_or_path'], "scheduler.pt")))

    try: #if args.fp16:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level='O1') #args.fp16_opt_level) For fp16: Apex AMP optimization level selected in ['O0', 'O1', 'O2', and 'O3']. See details at https://nvidia.github.io/apex/amp.html"

    # multi-gpu training (should be after apex fp16 initialization)
    if args['n_gpu'] > 1: model = torch.nn.DataParallel(model)

    # Train!
    logger.info("***** Running training *****")
    logger.info("  Num examples = %d", len(train_dataset))
    logger.info("  Num Epochs = %d", args['num_train_epochs'])
    logger.info("  Instantaneous batch size per GPU = %d", args['per_gpu_train_batch_size'])
    logger.info(
        "  Total train batch size (w. parallel, distributed & accumulation) = %d",
        args['train_batch_size'] * args['gradient_accumulation_steps'] * 1 #(torch.distributed.get_world_size() if args.local_rank != -1 else 1),
    )
    logger.info("  Gradient Accumulation steps = %d", args['gradient_accumulation_steps'])
    logger.info("  Total optimization steps = %d", t_total)

    global_step = 0
    epochs_trained = 0
    steps_trained_in_current_epoch = 0
    # Check if continuing training from a checkpoint
    if args['model_name_or_path'] and os.path.exists(args['model_name_or_path']):
        try:
            # set global_step to gobal_step of last saved checkpoint from model path
            checkpoint_suffix = args['model_name_or_path'].split("-")[-1].split("/")[0]
            global_step = int(checkpoint_suffix)
            epochs_trained = global_step // (len(train_dataloader) // args['gradient_accumulation_steps'])
            steps_trained_in_current_epoch = global_step % (len(train_dataloader) // args['gradient_accumulation_steps'])

            logger.info("  Continuing training from checkpoint, will skip to saved global_step")
            logger.info("  Continuing training from epoch %d", epochs_trained)
            logger.info("  Continuing training from global step %d", global_step)
            logger.info("  Will skip the first %d steps in the first epoch", steps_trained_in_current_epoch)
        except ValueError:
            logger.info("  Starting fine-tuning.")

    tr_loss, logging_loss = 0.0, 0.0

    model.zero_grad()
    train_iterator = trange(epochs_trained, int(args['num_train_epochs']), desc="Epoch", disable=False) #disable=args.local_rank not in [-1, 0]

    for _ in train_iterator:
        epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False, position=0, leave=True) #disable=args.local_rank not in [-1, 0] # # For coolab:position=0, leave=True
        for step, batch in enumerate(epoch_iterator):

            # Skip past any already trained steps if resuming training
            if steps_trained_in_current_epoch > 0:
                steps_trained_in_current_epoch -= 1
                continue

            inputs, labels = (batch, batch) # mask_tokens(batch, tokenizer, args) if args.mlm else (batch, batch) 
            inputs = inputs.to(args['device'])
            labels = labels.to(args['device'])
            model.train()
            outputs = model(inputs, labels=labels) #model(inputs, masked_lm_labels=labels) if args.mlm else model(inputs, labels=labels)
            loss = outputs[0]  # model outputs are always tuple in transformers (see doc)

            if args['n_gpu'] > 1:
                loss = loss.mean()  # mean() to average on multi-gpu parallel training
            if args['gradient_accumulation_steps'] > 1:
                loss = loss / args['gradient_accumulation_steps']

            with amp.scale_loss(loss, optimizer) as scaled_loss: #if args.fp16: else: loss.backward()
                scaled_loss.backward()
            
            tr_loss += loss.item()
            if (step + 1) % args['gradient_accumulation_steps'] == 0:
               
                torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args['max_grad_norm']) #if args.fp16: else: torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                # UserWarning: Seems like `optimizer.step()` has been overridden after learning rate scheduler initialization. Please, make sure to call `optimizer.step()` before `lr_scheduler.step()`.
                optimizer.step()
                scheduler.step()  # Update learning rate schedule
                model.zero_grad()
                global_step += 1

                if args['logging_steps'] > 0 and global_step % args['logging_steps'] == 0: #args.local_rank in [-1, 0] and 
                    # Log metrics
                    if (args['evaluate_during_training']):  # Only evaluate when single GPU otherwise metrics may not average well #  args.local_rank == -1 and 
                        results = evaluate(args, model, tokenizer)
                        for key, value in results.items():
                            tb_writer.add_scalar(f"eval_{key}", value, global_step)

                    # TODO CHECK UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`
                    tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)     
                    # print(scheduler.get_lr()[0])
                    # print(scheduler.get_last_lr())
                    # print(scheduler.get_last_lr()[0])
                    # tb_writer.add_scalar("lr", scheduler.get_last_lr(), global_step)     # Got <class 'list'>, but numpy array, torch tensor, or caffe2 blob name are expected.
                    tb_writer.add_scalar("loss", (tr_loss - logging_loss) / args['logging_steps'], global_step)
                    logging_loss = tr_loss

                if args['save_steps'] > 0 and global_step % args['save_steps'] == 0: # args.local_rank in [-1, 0] and 
                    checkpoint_prefix = "checkpoint"
                    # Save model checkpoint
                    output_dir = os.path.join(args['output_dir'], f"{checkpoint_prefix}-{global_step}")
                    os.makedirs(output_dir, exist_ok=True)
                    model_to_save = model #(model.module if hasattr(model, "module") else model)  # Take care of distributed/parallel training
                    model_to_save.save_pretrained(output_dir)
                    tokenizer.save_pretrained(output_dir)

                    torch.save(args, os.path.join(output_dir, "training_args.bin"))
                    logger.info("Saving model checkpoint to %s", output_dir)

                    _rotate_checkpoints(args, checkpoint_prefix) # Отключена ротация, т.е. удаление старых чекпоинтов, если их больше

                    torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
                    torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
                    logger.info("Saving optimizer and scheduler states to %s", output_dir)

            if 0 < args['max_steps'] < global_step:
                epoch_iterator.close()
                break
        if 0 < args['max_steps'] < global_step:
            train_iterator.close()
            break
    
    tb_writer.close() # if args.local_rank in [-1, 0]:

    return global_step, tr_loss / global_step


## 3.4 Загрузка исходной модели

In [19]:
# Проверки
if args['should_continue']:
  sorted_checkpoints = _sorted_checkpoints(args)
  if len(sorted_checkpoints) == 0:
    raise ValueError("Used --should_continue but no checkpoint was found in --output_dir.")
  else:
    print('Продолжаем с чекпоинта', sorted_checkpoints[-1])
    args['model_name_or_path'] = sorted_checkpoints[-1]

# Setup CUDA, GPU & distributed training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args['n_gpu'] = torch.cuda.device_count() if torch.cuda.is_available() else 0
args['device'] = device
print('Устройство', args['device'])

# Setup logging
logging.basicConfig(format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s", datefmt="%m/%d/%Y %H:%M:%S", level=logging.INFO)
logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    1, device, args['n_gpu'], False, True) #args['fp16']

config = AutoConfig.from_pretrained(args['model_name_or_path'], cache_dir=args['cache_dir'])
tokenizer = AutoTokenizer.from_pretrained(args['model_name_or_path'], cache_dir=args['cache_dir'])
args['block_size'] =  tokenizer.model_max_length if args['block_size'] <= 0 else min(args['block_size'], tokenizer.model_max_length)

model_trained = AutoModelForCausalLM.from_pretrained(args['model_name_or_path'], from_tf=bool(".ckpt" in args['model_name_or_path']),
                                            config=config, cache_dir=args['cache_dir'])
model_trained.to(args['device'])

06/10/2021 10:05:19 - WARNING - __main__ -   Process rank: 1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True


Продолжаем с чекпоинта comment_model/checkpoint-47000
Устройство cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


## 3.5 Обучение модели
**(!)ЕСЛИ ВЫВАЛИЛСЯ РАНТАЙМ(!)**
Обычно по памяти, нужно запустить пункты 2.1, 3.1.-3.4, пунткты 1.2-1.3 и 2.2а или 2.2б ЗАПУСКАТЬ НЕ НУЖНО.

In [20]:
# GPU
!nvidia-smi
# Обычная память
!free -h

Thu Jun 10 10:05:30 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    34W / 250W |   1465MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [21]:
# Загрузка обучающего датасета: или формируем токены для него, или загружаем дамп токенов, если ранее был сформирован
# Требуем много памяти на 300 тыс.записей общего датасета (240 тыс. обучающего) требуется примерно 12Гб.
logger.info("Training/evaluation parameters", args)
train_dataset = load_and_cache_examples(args, tokenizer, evaluate=False)

06/10/2021 10:05:36 - INFO - __main__ -   Training/evaluation parameters
06/10/2021 10:05:36 - INFO - __main__ -   Creating features from dataset file at 


Cчитали файл обучающего датасета: train.txt
Cконвертировали токены


06/10/2021 10:08:25 - INFO - __main__ -   Saving features into cached file gpt2_cached_lm_2048_train.txt


In [22]:
# Обучение модели
global_step, tr_loss = train(args, train_dataset, model_trained, tokenizer)
logger.info(" global_step = %s, average loss = %s", global_step, tr_loss)

06/10/2021 10:09:34 - INFO - __main__ -   ***** Running training *****
06/10/2021 10:09:34 - INFO - __main__ -     Num examples = 14163
06/10/2021 10:09:34 - INFO - __main__ -     Num Epochs = 5
06/10/2021 10:09:34 - INFO - __main__ -     Instantaneous batch size per GPU = 1
06/10/2021 10:09:34 - INFO - __main__ -     Total train batch size (w. parallel, distributed & accumulation) = 1
06/10/2021 10:09:34 - INFO - __main__ -     Gradient Accumulation steps = 1
06/10/2021 10:09:34 - INFO - __main__ -     Total optimization steps = 70815
06/10/2021 10:09:34 - INFO - __main__ -     Continuing training from checkpoint, will skip to saved global_step
06/10/2021 10:09:34 - INFO - __main__ -     Continuing training from epoch 3
06/10/2021 10:09:34 - INFO - __main__ -     Continuing training from global step 47000
06/10/2021 10:09:34 - INFO - __main__ -     Will skip the first 4511 steps in the first epoch
Iteration:  10%|▉         | 1389/14163 [00:00<00:00, 13882.90it/s]

Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


Iteration:  35%|███▌      | 5010/14163 [04:37<1:24:33,  1.80it/s]/usr/local/lib/python3.7/dist-packages/torch/optim/lr_scheduler.py:247: UserWarning: To get the last learning rate computed by the scheduler, please use `get_last_lr()`.
  warnings.warn("To get the last learning rate computed by the scheduler, "
Iteration:  39%|███▉      | 5510/14163 [09:14<1:19:56,  1.80it/s]06/10/2021 10:18:50 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-48000
06/10/2021 10:18:54 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-48000
Iteration:  46%|████▌     | 6510/14163 [18:33<1:10:42,  1.80it/s]06/10/2021 10:28:10 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-49000
06/10/2021 10:28:14 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-49000
Iteration:  48%|████▊     | 6782/14163 [21:09<1:07:47,  1.81it/s]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


Iteration:  53%|█████▎    | 7510/14163 [27:52<1:01:26,  1.80it/s]06/10/2021 10:37:29 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-50000
06/10/2021 10:37:33 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-50000
Iteration:  60%|██████    | 8510/14163 [37:11<52:15,  1.80it/s]06/10/2021 10:46:48 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-51000
06/10/2021 10:46:51 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-51000
Iteration:  67%|██████▋   | 9510/14163 [46:30<42:59,  1.80it/s]06/10/2021 10:56:07 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-52000
06/10/2021 10:56:11 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-52000
Iteration:  68%|██████▊   | 9672/14163 [48:05<41:17,  1.81it/s]

...

Iteration:  66%|██████▌   | 9347/14163 [1:27:05<44:30,  1.80it/s]06/10/2021 13:06:36 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-66000
06/10/2021 13:06:40 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-66000
Iteration:  66%|██████▌   | 9355/14163 [1:27:14<53:24,  1.50it/s]

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 65536.0


Iteration:  73%|███████▎  | 10347/14163 [1:36:24<35:13,  1.81it/s]06/10/2021 13:15:55 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-67000
06/10/2021 13:15:59 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-67000
Iteration:  80%|████████  | 11347/14163 [1:45:44<26:00,  1.80it/s]06/10/2021 13:25:15 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-68000
06/10/2021 13:25:19 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-68000
Iteration:  87%|████████▋ | 12347/14163 [1:55:03<16:46,  1.80it/s]06/10/2021 13:34:34 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-69000
06/10/2021 13:34:38 - INFO - __main__ -   Saving optimizer and scheduler states to comment_model/checkpoint-69000
Iteration:  94%|█████████▍| 13347/14163 [2:04:22<07:32,  1.80it/s]06/10/2021 13:43:53 - INFO - __main__ -   Saving model checkpoint to comment_model/checkpoint-

Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 131072.0


Epoch: 100%|██████████| 2/2 [3:41:54<00:00, 6657.18s/it]
06/10/2021 13:51:29 - INFO - __main__ -    global_step = 70815, average loss = 1.172627042677833


## Сохранение модели и расчет результирующих параметров
PS Во время обучения можно сохранять чекпоинт в консоли, например так

`zip -r -9 /content/gdrive/MyDrive/chats_emotions_and_voises/psy-chatbot/models/checkpoin50.zip ./comment_model/checkpoint-50000`

In [24]:
# Сохранение лучшей модели, чтобы можно было загрузить из предобученной
SAVE_MODEL_PATH = 'saved_model'
os.makedirs(SAVE_MODEL_PATH, exist_ok=True)

logger.info("Saving model checkpoint to %s", SAVE_MODEL_PATH)
# Save a trained model, configuration and tokenizer using `save_pretrained()`. They can then be reloaded using `from_pretrained()`
model_to_save = model_trained #(model.module if hasattr(model, "module") else model)  # Take care of distributed/parallel training
model_to_save.save_pretrained(SAVE_MODEL_PATH)
tokenizer.save_pretrained(SAVE_MODEL_PATH)
# Good practice: save your training arguments together with the trained model
torch.save(args, os.path.join(SAVE_MODEL_PATH, "training_args.bin"))
# Load a trained model and vocabulary that you have fine-tuned

model_loaded = AutoModelForCausalLM.from_pretrained(SAVE_MODEL_PATH)
# model = AutoModelWithLMHead.from_pretrained(SAVE_MODEL_PATH)
tokenizer = AutoTokenizer.from_pretrained(SAVE_MODEL_PATH)
model_loaded.to(args['device'])

06/10/2021 14:31:05 - INFO - __main__ -   Saving model checkpoint to saved_model


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50258, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )


In [25]:
# Сохраняем модель
!zip -r -9 $MODEL_PATH/saved_model.zip $SAVE_MODEL_PATH
%ls $MODEL_PATH

  adding: saved_model/ (stored 0%)
  adding: saved_model/config.json (deflated 50%)
  adding: saved_model/tokenizer_config.json (deflated 68%)
  adding: saved_model/tokenizer.json (deflated 77%)
  adding: saved_model/special_tokens_map.json (deflated 72%)
  adding: saved_model/merges.txt (deflated 78%)
  adding: saved_model/pytorch_model.bin (deflated 16%)
  adding: saved_model/training_args.bin (deflated 44%)
  adding: saved_model/vocab.json (deflated 75%)
  adding: saved_model/added_tokens.json (stored 0%)
checkpoin60.zip  checkpoint47.zip  saved_model47000.zip  saved_model.zip


In [ ]:
# Расчеты качества обучения
results = {}
checkpoints = [SAVE_MODEL_PATH] # args['output_dir']
args['eval_all_checkpoints'] = False
if args['eval_all_checkpoints'] : # вычислить значения всех чекпоинтов
    checkpoints = list(os.path.dirname(c) for c in sorted(glob.glob(SAVE_MODEL_PATH + "/**/" + WEIGHTS_NAME, recursive=True)))
    logging.getLogger("transformers.modeling_utils").setLevel(logging.WARN)  # Reduce logging
logger.info("Evaluate the following checkpoints: %s", checkpoints)
for checkpoint in checkpoints:
    global_step = checkpoint.split("-")[-1] if len(checkpoints) > 1 else ""
    prefix = checkpoint.split("/")[-1] if checkpoint.find("checkpoint") != -1 else ""
    #model = AutoModelWithLMHead.from_pretrained(checkpoint)
    model_evaluated = AutoModelForCausalLM.from_pretrained(checkpoint)
    model_evaluated.to(args['device'])
    result = evaluate(args, model_evaluated, tokenizer, prefix=prefix) # TODO перенести
    result = dict((k + f"_{global_step}", v) for k, v in result.items())
    results.update(result)
print(results)

06/10/2021 14:31:49 - INFO - __main__ -   Evaluate the following checkpoints: ['saved_model']
06/10/2021 14:31:51 - INFO - __main__ -   Creating features from dataset file at 


Cчитали файл обучающего датасета: test.txt


## Проверка модели

In [ ]:
# Проверка модели в целом, не в диалоговом формате. А в формате генерации текста
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=$SAVE_MODEL_PATH \
    --k=5 \
    --p=0.95 \
    --length=50

2021-06-07 07:55:05.163908: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
06/07/2021 07:55:12 - INFO - __main__ -   Namespace(device=device(type='cuda'), k=5, length=50, model_name_or_path='saved_model', model_type='gpt2', n_gpu=1, no_cuda=False, num_return_sequences=1, p=0.95, padding_text='', prompt='', repetition_penalty=1.0, seed=42, stop_token='</s>', temperature=1.0, xlm_language='')
Context >>> расскажи о психологии
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
расскажи о психологии, или о том, как ты ее понимаешь :)
|0|3|Вспомнила про это. Мне тоже было интересно узнать, как люди относятся к психологам и психологам-консультантам. Я сама психолог, но мне не очен
Context >>> работает вроде
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
ruGPT:
работает вроде все в порядке, но вот в последнее время я заметила, что мой муж стал очень часто делать мне укол